In [ ]:
%%capture
%pip install scipy numpy

In [3]:
import numpy as np
import scipy as sp

# **scipy.optimize**

## **minimize**

In [4]:
f_x = lambda x: x**2 + 4*x + 4
first_quess = [0]
result = sp.optimize.minimize(f_x, x0=first_quess)
display(result)
x = result['x'][0]
assert f_x(x) == 0
x

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.0
        x: [-2.000e+00]
      nit: 2
      jac: [ 0.000e+00]
 hess_inv: [[ 5.000e-01]]
     nfev: 6
     njev: 3

np.float64(-2.00000001888464)

In [5]:
f_x = lambda x: x[0]**2+x[1]**2
x0 = np.array([0, 0])
eq_cons = {'type': 'eq',
             'fun' : lambda x: np.array([-1 + x[0] + x[1]])
            }
ineq_cons = {'type': 'ineq',
             'fun' : lambda x: np.array([x[0], x[1]])
            }
result = sp.optimize.minimize(f_x, x0, method='SLSQP', constraints=[eq_cons, ineq_cons])
display(result)

result['x']

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.5
       x: [ 5.000e-01  5.000e-01]
     nit: 3
     jac: [ 1.000e+00  1.000e+00]
    nfev: 10
    njev: 3

array([0.5, 0.5])

## **Example 1: Solve a system of nonlinear equations**

Solve the system of equations:

$$
\begin{cases}
  x^2 + y^2 - 4 = 0 \\
  x - y = 0 
\end{cases}
$$

Use `scipy.optimize.fsolve` to solve the system. 
How would you approach this problem?


In [6]:
f_x = lambda x: [ 
    x[0]**2 + x[1]**2 - 4,
    x[0] - x[1]
]
sp.optimize.fsolve(f_x, [0,0])

array([1.41421356, 1.41421356])

## **Example 2: Linear Programming**

Minimize the objective function:  
$$
\min \, c^T x
$$

Subject to the constraints:  
$$
A \cdot x \leq b
$$

### **Problem Setup**

Minimize:  
$$
\min \, 2x_1 + 3x_2 
$$

Subject to:  
$$
\begin{cases}
  x_1 + 2x_2 \leq 20 \\
  2x_1 + x_2 \leq 20 \\
  x_1 \geq 0 \\
  x_2 \geq 0 \\
\end{cases}
$$

Use `scipy.optimize.linprog` to solve this problem. 
How would you formulate this problem?


In [7]:
c = [2,3]
A_ub = [ [1, 2], [2, 1] ]
b_ub = [ 20, 20 ]

sp.optimize.linprog(c, A_ub, b_ub, bounds=[(0, None),(0, None)], method='highs')

       message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
       success: True
        status: 0
           fun: 0.0
             x: [ 0.000e+00  0.000e+00]
           nit: 0
         lower:  residual: [ 0.000e+00  0.000e+00]
                marginals: [ 2.000e+00  3.000e+00]
         upper:  residual: [       inf        inf]
                marginals: [ 0.000e+00  0.000e+00]
         eqlin:  residual: []
                marginals: []
       ineqlin:  residual: [ 2.000e+01  2.000e+01]
                marginals: [-0.000e+00 -0.000e+00]

## **Example 3: Root-finding**

Find the root of the following equation:  
$$
\cos(x) - x = 0
$$

Use `scipy.optimize.root_scalar` to solve this problem.  
Which method would you use for this and why?


In [8]:
result = sp.optimize.root_scalar(lambda x: np.cos(x)-x, x0 = 0)
display(result)

sp.optimize.root_scalar(lambda x: np.cos(x)-x, bracket=[0, 1], method='brentq')

      converged: True
           flag: converged
 function_calls: 10
     iterations: 5
           root: 0.7390851332151607
         method: newton

      converged: True
           flag: converged
 function_calls: 8
     iterations: 7
           root: 0.7390851332151559
         method: brentq

## **Example 4: Global Optimization**

Find the global minimum of the following function:  
$$
f(x) = x^4 - 3x^3 + 2
$$

over the interval:  
$$
x \in [-3, 3]
$$

Use `scipy.optimize.differential_evolution` to solve this problem.  
Why is a global optimizer like `differential_evolution` needed for this problem?


In [9]:
bounds = [(-3, 3)]

sp.optimize.differential_evolution(lambda x: x**4 - 3*x**3 + 2, bounds)

             message: Optimization terminated successfully.
             success: True
                 fun: -6.542968749999989
                   x: [ 2.250e+00]
                 nit: 6
                nfev: 111
          population: [[ 2.250e+00]
                       [ 2.204e+00]
                       ...
                       [ 2.284e+00]
                       [ 2.241e+00]]
 population_energies: [-6.543e+00 -6.522e+00 ... -6.531e+00 -6.542e+00]
                 jac: [ 0.000e+00]

### 1. Local vs. Global Minimum
When you try to minimize a function, there are two possibilities:
- **Local Minimum**: A point where the function value is lower than its neighboring points, but not necessarily the lowest value in the entire range.
- **Global Minimum**: The absolute lowest point of the function across the entire search space.

---

### 2. Why Not Use Local Optimizers (like L-BFGS-B or SLSQP)?
Many local optimization methods, like gradient descent, Nelder-Mead, or L-BFGS-B, find a minimum starting from an initial guess.  
These methods follow the slope of the function downhill. However, if there are multiple "valleys" (local minima), the optimizer might get stuck in one of them instead of finding the deepest valley (global minimum).

**Example:**
$$
f(x) = x^4 - 3x^3 + 2
$$
This function has multiple local minima. If you start at \(x = -2\), you might get stuck in a local minimum near \(x = -1\) instead of finding the global minimum.

---

### 3. Why Use Differential Evolution?
- **Global Search**: Instead of starting from a single point, differential evolution searches the entire space. It doesn't "get stuck" in one place like gradient-based methods.  
- **Stochastic Approach**: It uses random mutation and recombination inspired by evolutionary biology, so it explores different regions of the space.  
- **No Need for Gradients**: It doesn't rely on derivatives, making it useful for non-smooth, non-convex, or complex functions with multiple minima.  

---

### 4. Key Takeaways
- If the function has **one minimum**, local optimizers like L-BFGS-B are fine.  
- If the function has **multiple minima**, global optimizers like **Differential Evolution** are needed.  
- Differential Evolution searches the **entire space**, whereas local methods follow the gradient, which might get "stuck" in local minima.

---


# **scipy.stats**

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind

### **Example 1**

### What Does the p-value Mean?

**Null Hypothesis ($H_0$)**: The average weights of apples from Farm A and Farm B are the same.
**Alternative Hypothesis ($H_A$)**: The average weights of apples from Farm A and Farm B are different.

If the **p-value** is small (typically less than 0.05), we reject the null hypothesis.

Here, , which is much larger than 0.05.
This means that there is not enough evidence to say that the weights from Farm A and Farm B are significantly different.

In [39]:
farm_a = [150, 155, 160, 165, 170]
farm_b = [155, 160, 165, 170, 175]

sp.stats.ttest_ind(farm_a, farm_b)

TtestResult(statistic=np.float64(-1.0), pvalue=np.float64(0.34659350708733416), df=np.float64(8.0))

### 🔥 Option 2: Chi-Square Test (Goodness of Fit)
#### 📝 **Problem Recap**
We have data on the favorite colors of 100 people.  
Here are the **observed frequencies**:
- **Red**: 20  
- **Blue**: 30  
- **Green**: 25  
- **Yellow**: 25  

The **expected frequencies** are based on the assumption that 25% of people prefer each color.  
Since there are 100 people, the expected frequency for each color is:  
$$
\text{Expected} = \frac{100}{4} = 25
$$
So, the expected distribution is:  
- **Red**: 25  
- **Blue**: 25  
- **Green**: 25  
- **Yellow**: 25  

---

### 🔥 1️⃣ **Hypotheses**
- **Null hypothesis** ($H_0$): The observed frequencies match the expected proportions.  
- **Alternative hypothesis** ($H_A$): The observed frequencies differ from the expected proportions.  

If the p-value is less than 0.05, we **reject the null hypothesis**, meaning the observed data does not fit the expected proportions.

---

### 🔥 2️⃣ **Chi-Square Statistic Formula**
The formula for the chi-square statistic is:  
$$
\chi^2 = \sum \frac{(O_i - E_i)^2}{E_i}
$$
Where:  
- $O_i$ = observed frequency for category $i$  
- $E_i$ = expected frequency for category $i$  

---

### 🔥 3️⃣ **Results**
After calculating the Chi-Square test, we get:  
- **Chi-Square Statistic** $\chi^2$ = 2.0  
- **p-value** = 0.5724  

---

### 🔥 4️⃣ **Interpretation**
1. **Chi-Square Statistic**:  
   The chi-square value tells us how far the observed frequencies are from the expected frequencies. A small value (like 2.0) means the observed and expected distributions are close to each other.

2. **p-value**:  
   The p-value is **0.5724**, which is larger than the standard significance level ($\alpha = 0.05$).  
   Since $p > 0.05$, we **fail to reject the null hypothesis**.  
   This means the observed data is **not significantly different** from the expected data.  

---

### 🔥 5️⃣ **Summary**
- The observed and expected frequencies for color preferences are close enough that we can't say they are statistically different.  
- **Conclusion**: The favorite colors of people in this survey align with the expected distribution.  


In [35]:
stat, pval = sp.stats.chisquare(f_obs=[20,30,25,25], f_exp=[25]*4)
f'Null Hypothesis {pval > 0.05}'

'Null Hypothesis True'

### 🔥 Option 7: Confidence Interval for the Mean
#### 📝 **Problem Recap**
You have a sample of 5 apple weights from a farm:  
[150, 155, 160, 165, 170]  

---

### 🔥 1️⃣ **What is a Confidence Interval?**
A confidence interval gives a range of values that likely contains the true population mean.  
For a **95% confidence interval**, we are 95% confident that the true population mean lies within this range.

The formula for a confidence interval for the mean is:  
$$
\text{CI} = \bar{x} \pm t^* \cdot \frac{s}{\sqrt{n}}
$$
Where:  
- $\bar{x}$ = sample mean  
- $t^*$ = critical t-value for 95% confidence and $n-1$ degrees of freedom  
- $s$ = sample standard deviation  
- $n$ = sample size  

---

### 🔥 2️⃣ **Calculation**
- **Sample data**: [150, 155, 160, 165, 170]  
- **Sample mean** ($\bar{x}$) = 160.0  
- **Sample size** ($n$) = 5  
- **Sample standard deviation** ($s$) = calculated using ddof=1 (sample standard deviation)  

Using `scipy.stats.t.interval`, the **95% confidence interval** is calculated as:  
- **Lower bound**: 150.18  
- **Upper bound**: 169.82  

The resulting confidence interval is:  
$$
[150.18, 169.82]
$$

---

### 🔥 3️⃣ **Interpretation**
1. **Confidence Interval**:  
   We are **95% confident** that the true mean weight of apples from this farm lies between **150.18 grams and 169.82 grams**.  

2. **Why is it a range?**  
   Since we only have a small sample of 5 apples, the confidence interval accounts for the uncertainty in our estimate of the true mean.  
   If we repeated this sampling process many times, 95% of the intervals would contain the true population mean.  

---

### 🔥 4️⃣ **Summary**
- **Sample mean**: 160.0 grams  
- **95% Confidence Interval**: [150.18, 169.82]  
- We are 95% confident that the true mean weight of apples lies in this range.  


In [36]:
# Sample data (apple weights)
data = [150, 155, 160, 165, 170]

# Calculate sample mean and standard error
sample_mean = np.mean(data)
sample_std = np.std(data, ddof=1)  # Use ddof=1 for sample standard deviation
n = len(data)  # Sample size

# Calculate the 95% confidence interval using scipy.stats.t.interval
confidence_level = 0.95
ci_low, ci_high = sp.stats.t.interval(confidence_level, df=n-1, loc=sample_mean, scale=sample_std / np.sqrt(n))

sample_mean, ci_low, ci_high


(np.float64(160.0),
 np.float64(150.1837841926122),
 np.float64(169.8162158073878))

# **scipy.linalg**

# **scipy.signal**

# **scipy.optimize.curve_fit**

# **scipy.sparse**

# **scipy.spatial.distance**